# Neural Network Exercise 5 (Elman & Jordan Networks)

Amirkabir University of Technology

Dr. Safabakhsh

By Gholamreza Dar 400131018

Spring 2022

## Imports

In [82]:
import os
import datetime

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Functions

### Data loading

In [83]:
def convert_label_to_binary(label):
    if label == 'normal' or label == 'ok':
        return 1
    else:
        return 0

In [84]:
def extract_data(file_path):
    '''Parse the data from 'file_path' and store the number lines in the segment list and the label lines in the labels list'''
    segments = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f.readlines():
            # Number line
            if line.startswith("\t"):
                segments.append(list(map(int, line.split())))
            # Whitespace
            elif line.startswith("\n"):
                pass
            # Label line
            else:
                labels.append(convert_label_to_binary(line[:-1]))

    return segments, labels


In [85]:
def load_data():
    '''Load the data from all 5 files and return a (463, 15, 6) numpy array and labels as a (463,) numpy array'''

    base_dir = "data"
    file_names = ["lp1.data", "lp2.data", "lp3.data", "lp4.data", "lp5.data"]
    
    X = []
    y = []

    # Loop through the files and extract the data and concatenate them
    for file_name in file_names:
        segments, labels = extract_data(os.path.join(base_dir, file_name))
        X += segments
        y += labels
    
    # Convert lists to numpy array and reshape them
    X = np.array(X)
    X = X.reshape(-1, 15, 6).astype(np.float32)
    y = np.array(y).astype(np.float32)
    
    return X, y

### Elman network

In [86]:
class ElmanModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(ElmanModel, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            state_t = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))

        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))
        return y

    # def evaluate(self, x, y, **kwargs):
    #     y_pred = self(x)
    #     return tf.keras.metrics.binary_accuracy(y, y_pred)

### Jordan Network

In [87]:
class JordanModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(JordanModel, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y_intermediate = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y_intermediate = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            hidden = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))
            y_intermediate = tf.keras.activations.tanh(tf.matmul(self.W_y_intermediate, tf.reshape(hidden, (self.hidden_units,1))) + tf.reshape(self.b_y_intermediate, (self.hidden_units,1)))
            state_t = y_intermediate
        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))

        return y
    
    # def evaluate(self, x, y):
    #     y_pred = self(x)
    #     return tf.keras.metrics.binary_accuracy(y, y_pred)

### Experiment functions

In [88]:
def train_elman(hidden_count, epochs=30, n_classes=2, lr=1e-3):
    elman_nn = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=n_classes)

    elman_nn.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    elman_nn.fit(
        X_train,
        y_train,
        batch_size=1,
        epochs=epochs,
        validation_data=(X_validation, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    
    # test_score = elman_nn.evaluate(X_test, y_test)
    test_score = 0.7

    return elman_nn, test_score

In [89]:
def train_jordan(hidden_count, epochs=30, n_classes=2, lr=1e-3):
    jordan_nn = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=n_classes)

    jordan_nn.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    jordan_nn.fit(
        X_train,
        y_train,
        batch_size=1,
        epochs=epochs,
        validation_data=(X_validation, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    
    # test_score = jordan_nn.evaluate(X_test, y_test)
    test_score=0.4

    return jordan_nn, test_score

### Ensemble Functions

In [90]:
def create_elman_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Elman submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        elman_submodel = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        elman_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=elman_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [91]:
def create_jordan_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Jordan submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        jordan_submodel = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        jordan_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=jordan_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [109]:
def create_ensemble_model(sub_models, active_index=None):
    '''This function takes a list of sub-models and returns an ensemble of them.
    Works with elman and jordan sub-models(and any combination of them).'''

    # We have to rename the layers to avoid duplicate layer names in the ensemble model
    for i, sub_model in enumerate(sub_models):
        for layer in sub_model.layers:
            layer._name = f'ensemble_{i+1}_{layer.name}'

    # Freeze every submodel except for the 'active_index' one(if 'active_index' is set)
    if active_index is not None:
        for i, sub_model in enumerate(sub_models):
            if i != active_index:
                for layer in sub_model.layers:
                    layer.trainable = False
            else:
                for layer in sub_model.layers:
                    layer.trainable = True
        
    # Create the ensemble model
    inputs = [sub_model.input for sub_model in sub_models]
    outputs = [sub_model.output for sub_model in sub_models]
    concat = tf.keras.layers.Concatenate()(outputs)
    ensemble_output = tf.keras.layers.Dense(1, activation='sigmoid')(concat)
    ensemble_output = tf.keras.layers.Flatten()(ensemble_output)
    ensemble_model = tf.keras.Model(inputs=inputs, outputs=ensemble_output)

    # Compile the ensemble model
    ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])

    return ensemble_model

In [93]:
def evaluate_ensemble(ensemble_model, X_test, y_test):
    '''Evaluates the ensemble model on the test set.'''
    # TODO NEEDS IMPLEMENTATION

### Ensemble Experiment Functions


In [94]:
def elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates elman sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = create_elman_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(elman_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(elman_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(elman_submodels))]
    X_test_ensemble = [X_test for i in range(len(elman_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test_ensemble, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [95]:
def jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    jordan_submodels = create_jordan_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(jordan_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(jordan_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(jordan_submodels))]
    X_test_ensemble = [X_test for i in range(len(jordan_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test_ensemble, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

In [96]:
def endgame_ensemble_experiment(elman_hidden_count_list=[10, 25, 50], jordan_hidden_count_list=[10, 25, 50]):
    '''Creates elman and jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = []
    if len(elman_hidden_count_list)>0:
        elman_submodels = create_elman_submodels(elman_hidden_count_list)

    jordan_submodels = []
    if len(jordan_hidden_count_list)>0:
        jordan_submodels = create_jordan_submodels(jordan_hidden_count_list)

    # Total count of sub-models
    total_submodels = len(elman_submodels) + len(jordan_submodels)

    # Split X_train for each sub-model 
    X_train_list = []
    for i in range(total_submodels):
        X_train_current = X_train[int(i*X_train.shape[0]/total_submodels):int((i+1)*X_train.shape[0]/total_submodels)]
        X_train_list.append(X_train_current)

    # Choose X_train and copy X_validation for all of the sub-models
    X_train_ensemble = [X_train_list[i] for i in range(total_submodels)]
    X_validation_ensemble = [X_validation for i in range(total_submodels)]
    X_test_ensemble = [X_test for i in range(total_submodels)]
    
    for i in range(total_submodels):
        # Create the mixed ensemble model and freeze every submodel except the i-th one
        ensemble_model = create_ensemble_model(elman_submodels + jordan_submodels, active_index=i)

        # Train the ensemble model
        ensemble_model.fit(
            X_train_ensemble,
            y_train,
            batch_size=1,
            epochs=30,
            validation_data=(X_validation_ensemble, y_validation),
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/endgame_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    # evaluate_ensemble(ensemble_model, X_test, y_test)
    test_accuracy = 0.6

    return ensemble_model, test_accuracy

## Main

### Loading the data

In [97]:
X, y = load_data()
print("X.shape:", X.shape)
print("X.dtype:", X.dtype)
print("y.shape:", y.shape)
print("y.dtype:", y.dtype)

X.shape: (463, 15, 6)
X.dtype: float32
y.shape: (463,)
y.dtype: float32


### Train, Test, validation split

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.125, random_state=1)

### Normalize Dataset?

In [99]:
# Normalize the data

### Experiments

#### Elman

In [100]:
elman_nn = ElmanModel(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

elman_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
elman_nn.fit(
    X_train,
    y_train,
    batch_size=1,
    epochs=30,
    validation_data=(X_validation, y_validation),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

Epoch 1/30
323/323 [==============================] - 3s 4ms/step - loss: 0.9185 - acc: 0.7276 - val_loss: 0.4288 - val_acc: 0.7447
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 0.4120 - acc: 0.7276 - val_loss: 0.3839 - val_acc: 0.7447
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3823 - acc: 0.7585 - val_loss: 0.3640 - val_acc: 0.7447
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3584 - acc: 0.7941 - val_loss: 0.3708 - val_acc: 0.7553
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3503 - acc: 0.8173 - val_loss: 0.3412 - val_acc: 0.8191
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3392 - acc: 0.8437 - val_loss: 0.3491 - val_acc: 0.8191
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3271 - acc: 0.8498 - val_loss: 0.3211 - val_acc: 0.8404
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
elman_nn.evaluate(X_test, y_test)

In [101]:
# Experiment with different hidden layer counts
elman_10, elman_10_score = train_elman(10)
elman_25, elman_25_score = train_elman(25)
elman_50, elman_50_score = train_elman(50)

Epoch 1/30
323/323 [==============================] - 3s 5ms/step - loss: 2.3142 - acc: 0.7337 - val_loss: 2.1663 - val_acc: 0.7447
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 2.2920 - acc: 0.7415 - val_loss: 2.1437 - val_acc: 0.7766
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2804 - acc: 0.7570 - val_loss: 2.1388 - val_acc: 0.7660
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2689 - acc: 0.7802 - val_loss: 2.1346 - val_acc: 0.7766
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2614 - acc: 0.7817 - val_loss: 2.1263 - val_acc: 0.7979
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2525 - acc: 0.7802 - val_loss: 2.1286 - val_acc: 0.7553
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2523 - acc: 0.7972 - val_loss: 2.1163 - val_acc: 0.7766
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 2.

In [102]:
print("elman_10_score:", elman_10_score)
print("elman_25_score:", elman_25_score)
print("elman_50_score:", elman_50_score)

elman_10_score: 0.7
elman_25_score: 0.7
elman_50_score: 0.7


#### Jordan

In [103]:
jordan_nn = JordanModel(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)

jordan_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])

jordan_nn.fit(
    X_train,
    y_train,
    batch_size=1,
    epochs=30,
    validation_data=(X_validation, y_validation),
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

Epoch 1/30
323/323 [==============================] - 3s 4ms/step - loss: 2.3574 - acc: 0.7276 - val_loss: 2.1731 - val_acc: 0.7447
Epoch 2/30
323/323 [==============================] - 1s 2ms/step - loss: 1.0798 - acc: 0.7198 - val_loss: 0.3887 - val_acc: 0.7234
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3848 - acc: 0.7353 - val_loss: 0.3648 - val_acc: 0.7766
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3583 - acc: 0.7724 - val_loss: 0.3387 - val_acc: 0.7979
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3408 - acc: 0.8158 - val_loss: 0.3472 - val_acc: 0.8085
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3358 - acc: 0.8158 - val_loss: 0.3267 - val_acc: 0.8191
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3255 - acc: 0.8251 - val_loss: 0.3182 - val_acc: 0.8191
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
jordan_nn.evaluate(X_test, y_test)

In [104]:
jordan_10, jordan_10_score = train_jordan(10)
jordan_25, jordan_25_score = train_jordan(25)
jordan_50, jordan_50_score = train_jordan(50)

Epoch 1/30
323/323 [==============================] - 3s 4ms/step - loss: 2.5873 - acc: 0.7276 - val_loss: 2.2141 - val_acc: 0.7447
Epoch 2/30
323/323 [==============================] - 1s 2ms/step - loss: 2.3226 - acc: 0.7276 - val_loss: 2.1741 - val_acc: 0.7447
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 2.3022 - acc: 0.7276 - val_loss: 2.1598 - val_acc: 0.7447
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2894 - acc: 0.7353 - val_loss: 2.1504 - val_acc: 0.7660
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2819 - acc: 0.7616 - val_loss: 2.1473 - val_acc: 0.7660
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2752 - acc: 0.7678 - val_loss: 2.1396 - val_acc: 0.7766
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 2.2707 - acc: 0.7740 - val_loss: 2.1333 - val_acc: 0.7872
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 2.

In [105]:
print("jordan_10_score:", jordan_10_score)
print("jordan_25_score:", jordan_25_score)
print("jordan_50_score:", jordan_50_score)

jordan_10_score: 0.4
jordan_25_score: 0.4
jordan_50_score: 0.4


### Ensemble

#### Elman Only 

In [110]:
elman_only_ensemble, elman_only_ensemble_score = elman_only_ensemble_experiment(
    hidden_count_list=[10, 25, 50])

Epoch 1/30
323/323 [==============================] - 5s 6ms/step - loss: 0.6208 - acc: 0.6734 - val_loss: 0.5153 - val_acc: 0.7553
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 0.4748 - acc: 0.7972 - val_loss: 0.4240 - val_acc: 0.7979
Epoch 3/30
323/323 [==============================] - 1s 3ms/step - loss: 0.4111 - acc: 0.8158 - val_loss: 0.3779 - val_acc: 0.8298
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3707 - acc: 0.8220 - val_loss: 0.4045 - val_acc: 0.7447
Epoch 5/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3533 - acc: 0.8189 - val_loss: 0.3564 - val_acc: 0.8723
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3217 - acc: 0.8653 - val_loss: 0.3300 - val_acc: 0.8723
Epoch 7/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3533 - acc: 0.8390 - val_loss: 0.3641 - val_acc: 0.8617
Epoch 8/30
323/323 [==============================] - 1s 3ms/step - loss: 0.

In [114]:
elman_only_ensemble(X_test)

ValueError: Layer "model_9" expects 3 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor: shape=(93, 15, 6), dtype=float32, numpy=
array([[[  46.,   29.,   59.,  -98.,  125.,  -16.],
        [  19.,   18.,   96.,  -42.,   10.,   -6.],
        [  27.,   23.,   93.,  -42.,   31.,   -9.],
        ...,
        [  -4.,   -7.,   84.,   -2.,  -16.,   -8.],
        [  -3.,   -3.,   80.,   -7.,  -13.,   -1.],
        [   0.,  -15.,   86.,   13.,  -11.,   -1.]],

       [[  -4.,    3.,   76.,  -11.,  -16.,   -5.],
        [  -2.,  -13.,   91.,   18.,  -12.,   -5.],
        [   0.,    0.,   71.,   -9.,   -6.,   -4.],
        ...,
        [  -4.,   -1.,   83.,   -7.,  -18.,  -13.],
        [   2.,   -9.,   83.,    7.,   -5.,   -6.],
        [   1.,    0.,   74.,   -6.,   -8.,    3.]],

       [[  -1.,    2.,    4.,   -1.,   -7.,    0.],
        [  -1.,    1.,    6.,    5.,   -6.,    0.],
        [   2.,    0.,   17.,    3.,   -4.,    0.],
        ...,
        [   1.,    0.,   

#### Jordan Only

In [111]:
jordan_only_ensemble, jordan_only_ensemble_score = jordan_only_ensemble_experiment(
    hidden_count_list=[10, 25, 50])

Epoch 1/30
323/323 [==============================] - 7s 7ms/step - loss: 0.5490 - acc: 0.7508 - val_loss: 0.4353 - val_acc: 0.7872
Epoch 2/30
323/323 [==============================] - 1s 4ms/step - loss: 0.4090 - acc: 0.8050 - val_loss: 0.3723 - val_acc: 0.8085
Epoch 3/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3569 - acc: 0.8328 - val_loss: 0.3263 - val_acc: 0.8191
Epoch 4/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3337 - acc: 0.8266 - val_loss: 0.3420 - val_acc: 0.7979
Epoch 5/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3396 - acc: 0.8251 - val_loss: 0.3311 - val_acc: 0.8085
Epoch 6/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3129 - acc: 0.8251 - val_loss: 0.2954 - val_acc: 0.8298
Epoch 7/30
323/323 [==============================] - 1s 3ms/step - loss: 0.3175 - acc: 0.8297 - val_loss: 0.3251 - val_acc: 0.7872
Epoch 8/30
323/323 [==============================] - 1s 3ms/step - loss: 0.

#### Mixed

In [112]:
mixed_ensemble, mixed_ensemble_score = endgame_ensemble_experiment(
    elman_hidden_count_list=[10, 25, 50],
    jordan_hidden_count_list=[10, 25, 50])

ValueError: Data cardinality is ambiguous:
  x sizes: 53, 54, 54, 54, 54, 54
  y sizes: 323
Make sure all arrays contain the same number of samples.

## Tensorboard

In [ ]:
# load tensorboard extension
%load_ext tensorboard

# start tensorboard in the 'logs' directory
%tensorboard --logdir=logs/